In [9]:
%matplotlib qt
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import time

import ssm_timeSeries as ts  # my self-written time series overhead
import ssm_fit               # my self-written library for state-space model fitting

from IPython import display  # for live plotting

import random
from datetime import datetime     # generate random seed for 
random.seed(datetime.now())       # np.random. Once this is fixed, all 
rngSeed = random.randint(0, 1000) # other 'randomness' is also fixed

from scipy.io import savemat # store results for comparison with Matlab code

xDim = 5
yDim = 15

T = 500000
Trial = 1


subpops = [list(range(7,yDim)), list(range(0,yDim-7))]
obsTime = [int(round(T/4)), int(round(2*T/3)), T]
obsPops = [0,1,0]
obsScheme = {'subpops': subpops,
             'obsTime': obsTime,
             'obsPops': obsPops}
[obsIdxG, idxgrps] = ssm_fit._computeObsIndexGroups(obsScheme,yDim)
obsScheme['obsIdxG'] = obsIdxG # add index groups and 
obsScheme['idxgrps'] = idxgrps # their occurences        


while True:
    W    = np.random.normal(size=[xDim,xDim])
    if np.abs(np.linalg.det(W)) > 0.001:
        break
A    = np.diag(np.linspace(0.1,0.95,xDim))  #np.diag(np.random.uniform(size=[xDim]))
A    = np.dot(np.dot(W, A), np.linalg.inv(W))
Qnoise = np.random.normal(size=[xDim,xDim])/9
Qnoise = np.dot(Qnoise, Qnoise.transpose())
Q    = np.identity(xDim) + Qnoise
mu0  = np.random.normal(size=[xDim]) #np.random.normal(size=[xDim])
V0   = np.identity(xDim)
C = np.random.normal(size=[yDim, xDim])
R = np.identity(yDim) + np.diag(np.abs(np.random.normal(size=[yDim])))
R[0,0] = 5
R[1,1] = 3
R[2,2] = 9
R[3,3] = 4

d  = np.arange(yDim)
d -= 10

uDim = 1
B = np.random.normal(size=[xDim,uDim])
u = 5 * np.random.normal(size=[uDim, T, Trial])

seq = ts.setStateSpaceModel('iLDS', [xDim,yDim,uDim], [A,B,Q,mu0,V0,C,d,R])  # initiate true model
seq.giveEmpirical().addData(Trial,T,[u],rngSeed)                             # draw toy data

# check implementation of E- and M- step by doing one such step each:
x = seq.giveEmpirical().giveData().giveTracesX()
y = seq._empirical._data.giveTracesY()          

plt.plot(x[0,range(300),0])

no data model. Assuming data is given by real experiment or other external source
checking factorization of model ...
... factorization (locally) describes directed acyclic graph


In [10]:
%matplotlib qt
# initiate parameters for analysis model
A_0   = np.diag(np.random.uniform(size=[xDim]))
Q_0   = 50*np.identity(xDim)              
mu0_0 = 5*np.random.normal(size=[xDim])   
V0_0  = 50*np.identity(xDim)               
C_0   = np.random.normal(size=[yDim,xDim]) 
d_0   = np.random.normal(size=yDim)
d_0    = np.mean(y,(1,2))          
R_0   = 50*np.identity(yDim)    
B_0   = 0 * np.random.normal(size=[xDim, uDim])
                                                   
[Ext, Extxt, Extxtm1,LL0]    = ssm_fit._LDS_E_step(A_0,B_0,Q_0,mu0_0,V0_0,C_0,d_0,R_0, 
                                                   y,u,obsScheme)    # do one E-step 

[A_1,B_1,Q_1,mu0_1,V0_1,C_1,d_1,R_1,my,syy,suu,suuinv,Ti] = ssm_fit._LDS_M_step(Ext, 
                                                                                Extxt, 
                                                                                Extxtm1, 
                                                                                y, u,
                                                                                obsScheme) # do one M-step
[Ext1, Extxt1, Extxtm11,LL1] = ssm_fit._LDS_E_step(A_1,B_1,Q_1,mu0_1,V0_1,C_1,d_1,R_1, 
                                                   y,u,obsScheme)   # do another E-step

[Ext_true, Extxt_true, Extxtm1_true, LLtr] = ssm_fit._LDS_E_step(A,B,Q,mu0,V0,C,d,R,
                                                                 y,u,obsScheme)


subpops
[[7, 8, 9, 10, 11, 12, 13, 14], [0, 1, 2, 3, 4, 5, 6, 7]]
A
[[ 0.71234391 -0.01505518 -0.17056183 -0.13052147 -0.19958856]
 [-0.01332147  0.58323216 -0.02558483 -0.03297903 -0.22259436]
 [-0.24408428  0.1507289   0.3989222  -0.00765515 -0.0171514 ]
 [-0.20894597  0.11095391  0.11229339  0.44081384  0.01248147]
 [-0.32476986 -0.13934105 -0.04247226 -0.13089869  0.42130303]]
B
[[ 0.3393187 ]
 [ 0.38646332]
 [ 0.20148843]
 [ 0.11267582]
 [-0.24058292]]
Q
[[ 24.01211874  10.58150625 -12.51013911  -7.36592918  -8.1785511 ]
 [ 10.58150625  22.34525239  -6.03352804  -1.91726741 -10.14989303]
 [-12.51013911  -6.03352804  27.4361241    6.46646578   2.70316562]
 [ -7.36592918  -1.91726741   6.46646578  16.01787014   0.65998299]
 [ -8.1785511  -10.14989303   2.70316562   0.65998299  23.60888815]]
C
[[ 0.38288114 -0.43751516 -0.41611309 -0.18832293 -0.26288877]
 [-0.63526387 -0.16685524 -0.42142973 -0.01766123  0.14568898]
 [-0.90824909  0.74910868  0.06126748 -0.32180524  0.61999829]
 [ 0

In [ ]:

# initiate parameters for analysis model
A_0   = np.diag(np.random.uniform(size=[xDim]))
Q_0   = 50*np.identity(xDim)              
mu0_0 = 5*np.random.normal(size=[xDim])   
V0_0  = 50*np.identity(xDim)               
C_0   = np.random.normal(size=[yDim,xDim]) 
d_0   = np.random.normal(size=yDim)
d_0   = np.mean(y,(1,2))          
R_0   = 50*np.identity(yDim)    
B_0   = 0 * np.random.normal(size=[xDim, uDim])


seq = ts.setStateSpaceModel('iLDS', [xDim,yDim,uDim], [A_0,B_0,Q_0,mu0_0,V0_0,C_0,d_0,R_0], seq) # adds analysis model 
model = seq.analysis1.giveModel()                                                     

t = time.time()
%matplotlib inline
LLs = seq.analysis1.fitModel(30,           # maxIter                             
                             np.log(1.00001), # stop if likelihood change < 0.001%
                             None,
                             0, 
                             None, 
                             None,
                             obsScheme,
                             True,
                             True)
elapsedTime = time.time() - t
print('elapsed time for fitting is')
print(elapsedTime)
[A_h,B_h,Q_h,mu0_h,V0_h,C_h,d_h,R_h] = model.givePars().copy()

[Ext_h, Extxt_h, Extxtm1_h, LL_h]          = ssm_fit._LDS_E_step(A_h,B_h,Q_h,mu0_h,V0_h,C_h,d_h,R_h, 
                                                                 y,u,obsScheme)


Pi_h    = np.array([sp.linalg.solve_discrete_lyapunov(A_h, Q_h)])[0,:,:]
Pi_t_h  = np.dot(A_h.transpose(), Pi_h)

Pi    = np.array([sp.linalg.solve_discrete_lyapunov(A, Q)])[0,:,:]
Pi_t  = np.dot(A.transpose(), Pi)

dataCov  = np.cov(y[:,0:T-1,0], y[:,1:T,0])
covyy    = dataCov[np.ix_(np.arange(0, yDim), np.arange(0,     yDim))]
covyy_m1 = dataCov[np.ix_(np.arange(0, yDim), np.arange(yDim,2*yDim))]

%matplotlib qt
plt.figure(1)
cmap = matplotlib.cm.get_cmap('brg')
clrs = [cmap(i) for i in np.linspace(0, 1, xDim)]
for i in range(xDim):
    plt.subplot(xDim,1,i)
    plt.plot(x[i,:,0], color=clrs[i])
    plt.hold(True)
    if np.mean( np.square(x[i,:,0] - Ext_h[i,:,0]) ) < np.mean( np.square(x[i,:,0] + Ext_h[i,:,0]) ):
        plt.plot( Ext_h[i,:,0], color=clrs[i], ls=':')
    else:
        plt.plot(-Ext_h[i,:,0], color=clrs[i], ls=':')
m = covyy.min()
M = covyy.max()        
plt.figure(2)
plt.subplot(1,3,1)
plt.imshow(np.dot(np.dot(C_h, Pi_h), C_h.transpose()) + R_h, interpolation='none')
plt.title('cov_hat(y_t,y_t)')
plt.clim(m,M)
plt.subplot(1,3,2)
plt.imshow(covyy,    interpolation='none')
plt.title('cov_emp(y_t,y_t)')
plt.clim(m,M)
plt.subplot(1,3,3)
plt.imshow(np.dot(np.dot(C, Pi), C.transpose()) + R, interpolation='none')
plt.title('cov_true(y_t,y_t)')
plt.clim(m,M)
plt.figure(3)

m = covyy_m1.min()
M = covyy_m1.max()        
plt.subplot(1,3,1)
plt.imshow(np.dot(np.dot(C_h, Pi_t_h), C_h.transpose()), interpolation='none')
plt.title('cov_hat(y_t,y_{t-1})')
plt.clim(m,M)
plt.subplot(1,3,2)
plt.imshow(covyy_m1,    interpolation='none')
plt.title('cov(y_t,y_{t-1})')
plt.clim(m,M)
plt.subplot(1,3,3)
plt.imshow(np.dot(np.dot(C, Pi_t), C.transpose()), interpolation='none')
plt.title('cov_true(y_t,y_{t-1})')
plt.clim(m,M)
plt.figure(4)
plt.plot(np.sort(np.linalg.eig(A)[0]), 'r')
plt.hold(True)
plt.plot(np.sort(np.linalg.eig(A_h)[0]), 'b')
plt.legend(['true', 'est'])


#matlabSaveFile = {'x': x, 'y': y, 'u' : u, 
#                  'A':A, 'B':B, 'Q':Q, 'mu0':mu0,'V0':V0,'C':C,'d':d,'R':R,
#                  'A_0':A_0, 'B_0':B_0, 'Q_0':Q_0, 'mu0_0':mu0_0,'V0_0':V0_0,'C_0':C_0,'d_0':d_0,'R_0':R_0,
#                  'A_1':A_1, 'B_1':B_1, 'Q_1':Q_1, 'mu0_1':mu0_1,'V0_1':V0_1,'C_1':C_1,'d_1':d_1,'R_1':R_1,
#                  'A_h':A_h, 'Q_h':Q_h, 'mu0_h':mu0_h,'V0_h':V0_h,'C_h':C_h,'R_h':R_h,
#                  'Ext':Ext, 'Extxt':Extxt, 'Extxtm1':Extxtm1,
#                  'T':T,
#                  'LL0': LL0, 'LL1': LL1}
#savemat('LDS_data.mat',matlabSaveFile)



In [ ]:
plotRange = range(2000, 2200)
plt.subplot(1,2,1)
plt.plot((Ext_true[0,plotRange,0] - np.mean(Ext_true[0,plotRange,0]))/np.std(Ext_true[0,plotRange,0]))
plt.plot(x[0,plotRange,0]/np.std(x[0,plotRange,0]))
plt.xlabel('#iter')
plt.ylabel('x_t')
plt.legend(['E[x] with true params', 'x true'])
plt.title('z-scored')
plt.subplot(1,2,2)
plt.plot(Ext_true[0,plotRange,0])
plt.plot(x[0,plotRange,0])
plt.xlabel('#iter')
plt.ylabel('x_t')
plt.legend(['E[x] with true params', 'x true'])
plt.title('raw')
print('offset')
print(np.mean(Ext_true[0,plotRange,0]))

In [ ]:
plotRange = range(2000, 2500)
plt.subplot(1,2,1)
plt.plot(-(Ext1[0,plotRange,0] - np.mean(Ext1[0,plotRange,0]))/np.std(Ext1[0,plotRange,0]))
plt.plot(x[0,plotRange,0]/np.std(x[0,plotRange,0]))
plt.xlabel('#iter')
plt.ylabel('x_t')
plt.legend(['E[x] after first iteration', 'x true'])
plt.title('z-scored')
plt.subplot(1,2,2)
plt.plot(-Ext1[0,plotRange,0])
plt.plot(x[0,plotRange,0])
plt.xlabel('#iter')
plt.ylabel('x_t')
plt.legend(['E[x] after first iteration', 'x true'])
plt.title('raw')
print('offset')
print(np.mean(Ext1[0,plotRange,0]))


In [ ]:
plotRange = range(2000, 2200)

i = 0
plt.subplot(1,2,1)
plt.plot(-(Ext_h[i,plotRange,0] - np.mean(Ext_h[i,plotRange,0]))/np.std(Ext_h[i,plotRange,0]))
plt.plot(x[i,plotRange,0]/np.std(x[i,plotRange,0]))
plt.xlabel('#iter')
plt.ylabel('x_t')
plt.legend(['E[x] after last iteration', 'x true'])
plt.title('z-scored')
plt.subplot(1,2,2)
plt.plot(-Ext_h[i,plotRange,0])
plt.plot(x[i,plotRange,0])
plt.xlabel('#iter')
plt.ylabel('x_t')
plt.legend(['E[x] after last iteration', 'x true'])
plt.title('raw')
print('offset')
print(np.mean(Ext_h[i,plotRange,0]))

print('scale [in E[xt]]')
print(np.std(Ext_h[i,plotRange,0])/np.std(x[i,plotRange,0]))


In [ ]:
plotRange = range(2000, 2500)

#plt.plot(Ext_true[0,plotRange,0])
plt.plot(Ext_h[0,plotRange,0])
plt.plot(x[0,plotRange,0])
plt.xlabel('#iter')
plt.ylabel('x_t')
print('A_0')
print(A_0)
print('A')
print(A)
print('A_h')
print(A_h)
print('B_0')
print(B_0)
print('B')
print(B)
print('B_h')
print(B_h)
print('Q_0')
print(Q_0)
print('Q')
print(Q)
print('Q_h')
print(Q_h)

In [ ]:
plt.subplot(1,4,1)
plt.title('C_0')
plt.imshow(C_0, interpolation='none')
plt.subplot(1,4,2)
plt.title('C')
plt.imshow(C, interpolation='none')
plt.subplot(1,4,3)
plt.title('C_h')
plt.imshow(-C_h, interpolation='none')
plt.subplot(1,4,4)
plt.title('C_h / C')
plt.imshow(C / C_h, interpolation='none')
plt.colorbar()

In [ ]:
plt.subplot(1,4,1)
plt.title('d_0')
plt.imshow(d_0.reshape(yDim,1), interpolation='none')
plt.subplot(1,4,2)
plt.title('d')
plt.imshow(d.reshape(yDim,1), interpolation='none')
plt.subplot(1,4,3)
plt.title('d_h')
plt.imshow(d_h.reshape(yDim,1), interpolation='none')
plt.subplot(1,4,4)
plt.title('d_h / d')
plt.imshow((d_h / d).reshape(yDim,1), interpolation='none')
plt.colorbar()
print(d_0)
print(d)
print(d_h)

In [ ]:
plt.subplot(1,4,1)
plt.title('R_0')
plt.imshow(R_0.diagonal().reshape(yDim,1), interpolation='none')
plt.subplot(1,4,2)
plt.title('R')
plt.imshow(R.diagonal().reshape(yDim,1), interpolation='none')
plt.subplot(1,4,3)
plt.title('R_h')
plt.imshow(R_h.diagonal().reshape(yDim,1), interpolation='none')
plt.subplot(1,4,4)
plt.title('R_h / R')
plt.imshow((R_h.diagonal() / R.diagonal()).reshape(yDim,1), interpolation='none')
plt.colorbar()


In [ ]:
plt.plot(LLs)

In [ ]:
i = 10
plotRange = range(8000, 8500)
#plt.plot(np.dot(C[i,:], Ext_true[:,plotRange,0]) +d[i])
plt.plot(np.dot(C_h[i,:], Ext_h[:,plotRange,0]) +d_h[i])
plt.plot(np.dot(C[i,:], x[:,plotRange,0]) +d[i])
#plt.plot(y[i,plotRange,0])

print('std')
print(np.std(y[i,plotRange,0]))
print(np.std(np.dot(C[i,:], x[:,plotRange,0]) +d[i]))


print('std ratio')
print(np.std(y[i,plotRange,0])/np.std(np.dot(C[i,:], x[:,plotRange,0]) +d[i]))

print('means')
print(np.mean(y[i,plotRange,0]))
print(np.mean(np.dot(C[i,:], Ext_true[:,plotRange,0]) +d[i]))

res = np.dot(C_h[i,:], Ext_h[:,plotRange,0]) +d_h[i] - y[i,plotRange,0]
print(np.mean(res))
print(np.var(res))
print(R.diagonal()[i])


In [ ]:
print('B')
print(B)
print('B_h')
print(B_h)

In [7]:
matlabSaveFile = {'x': x, 'y': y, 'u' : u, 
                  'A':A, 'B':B, 'Q':Q, 'mu0':mu0,'V0':V0,'C':C,'d':d,'R':R,
                  'A_0':A_0, 'B_0':B_0, 'Q_0':Q_0, 'mu0_0':mu0_0,'V0_0':V0_0,'C_0':C_0,'d_0':d_0,'R_0':R_0,
                  'A_1':A_1, 'B_1':B_1, 'Q_1':Q_1, 'mu0_1':mu0_1,'V0_1':V0_1,'C_1':C_1,'d_1':d_1,'R_1':R_1,
                  'A_h':A_h, 'Q_h':Q_h, 'mu0_h':mu0_h,'V0_h':V0_h,'C_h':C_h,'R_h':R_h,
                  'Ext':Ext, 'Extxt':Extxt, 'Extxtm1':Extxtm1,
                  'T':T,
                  'LL0': LL0, 'LL1': LL1}
savemat('LDS_data.mat',matlabSaveFile)
